In [1]:
import json
from openai import OpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored 
import pandas as pd

GPT_MODEL = "text-embedding-3-large"
client = OpenAI()

In [ ]:
promotion_df = pd.read_excel(
    "../data/Recommend Promotion Case.xlsx", sheet_name="Promotion Special Day KBANK"
).reset_index()

promotion_df.to_json("../data/promotions.json", orient="records", force_ascii=False)

In [2]:
def get_embedding(text, model):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

In [3]:
promotions = json.load(open("../data/promotions.json"))

In [4]:
embedded_promotions = []
for promotion in promotions:
    embedded_promotion = {}
    for key, value in promotion.items():
        if key != 'index':
            embedded_promotion[key] = get_embedding(value, GPT_MODEL)
        else:
            embedded_promotion[key] = value
    embedded_promotions.append(embedded_promotion)

In [5]:
file_path = "../data/embed_promotions.json"

# Save the list of dictionaries as JSON
with open(file_path, "w") as json_file:
    json.dump(embedded_promotions, json_file)